In [ ]:
#!pip uninstall tensorflow

In [ ]:
#!pip install tensorflow-gpu==2.0.0

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import layers
print(tf.__version__)
import time
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches
from sklearn.metrics import roc_auc_score
import random
import itertools
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from sklearn.metrics import confusion_matrix


2.3.0


In [2]:
size = 25

In [3]:
'''config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
tf.compat.v1.Session(config=config)'''

'config = tf.compat.v1.ConfigProto()\nconfig.gpu_options.allow_growth = True\nsess = tf.compat.v1.Session(config=config)\nconfig = tf.compat.v1.ConfigProto()\nconfig.gpu_options.per_process_gpu_memory_fraction = 0.95\ntf.compat.v1.Session(config=config)'

![title](擷取.PNG)

In [4]:
def gernate_data(n, nu, seed, range_, multi, size):
    ## normal
    np.random.seed(seed=seed)
    y_train = np.ones((int(n*(1-nu)),))
    mean = np.random.uniform(-5, 5, size)
    cov_  = np.random.uniform(0, 1, (size, size))* 3
    cov = np.dot(cov_.T,cov_)
    x_train = np.random.multivariate_normal(mean, cov, (int(n*(1-nu))), check_valid  = 'raise')
    x_train = x_train.astype('float32')

    
    
    np.random.seed(seed=seed)
    x_trains_ = None ## normal + outlier
    for i in range(multi):
        mean = np.random.uniform(-range_, range_, size)
        cov_ = np.random.randn(size, size)
        cov = np.dot(cov_.T,cov_)
        x_train_ = np.random.multivariate_normal(mean, cov, (int(n*nu/multi)), check_valid  = 'raise')
        try:
            x_trains_ = np.vstack((x_trains_, x_train_))
        except:
            x_trains_ = x_train_
    ## 畫圖
    '''fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x_train[:, 0], x_train[:, 1],x_train[:, 2], s=0.9, label = 'normal')
    ax.scatter(x_trains_[:,0], x_trains_[:,1], x_trains_[:,2], s=0.9, label = 'outlier')
    plt.legend()
    plt.title('normal + outlier')
    plt.show()'''

    y_trains_ = np.zeros(((len(x_trains_),)))
    x_train_mix = np.vstack((x_trains_, x_train)).astype('float32')
    y_train_mix = np.hstack((y_trains_, y_train)).astype('float32')
    
    return x_train_mix, y_train_mix

## 參數設定

In [5]:

h_dim = [10, 5]
regular = 1e-5
max_lr = 1e-6
base_lr = 1e-5


n_epochs = 1000

optimizer = tf.keras.optimizers.Adam(lr = base_lr)

In [6]:
def make_ae_model():

    inputs = tf.keras.Input(shape=(size,), name='inputs')

    x = tf.keras.layers.Dense(h_dim[0], activation='linear', use_bias=False)(inputs)
    x = tf.keras.layers.Dense(h_dim[1], activation='linear', use_bias=True, name = 'hidden')(x)
    x = tf.keras.layers.Dense(h_dim[0], activation='linear', use_bias=False)(x)
    x = tf.keras.layers.Dense(size, activation='linear', use_bias=False)(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

ae_model = make_ae_model()
ae_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                250       
_________________________________________________________________
hidden (Dense)               (None, 5)                 55        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 25)                250       
Total params: 605
Trainable params: 605
Non-trainable params: 0
_________________________________________________________________


In [7]:
def make_model(model):
    
    model.trainable = True
    x = model.get_layer('hidden').output

    model_out = tf.keras.Model(inputs = model.input, outputs = x)
    
    return model_out

model = make_model(ae_model)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                250       
_________________________________________________________________
hidden (Dense)               (None, 5)                 55        
Total params: 305
Trainable params: 305
Non-trainable params: 0
_________________________________________________________________


## Train SA-SVDD

In [66]:
class SA_DeepSVDD_Trainer():
    
    def __init__(self, nu, SVDD_model, n_samples):
        self.SVDD_model = SVDD_model
        self.n_samples = n_samples
        self.nu = nu
        self.C = 1/(n_samples * nu)
        
        ## 設置lagrange因子、Radius
        self.radius = tf.Variable(0.1, dtype=tf.float32, name="radius2", trainable=True)
        self.optimizer = tf.keras.optimizers.Adam(lr = 1e-3, epsilon = 1e-6)

        
    def get_radius(self, batch_x):
        ## Optimally solve for radius R via the (1-nu)-quantile of distances.
 
        modeloutput = self.SVDD_model(batch_x, training=False)

        distance = tf.reduce_sum(tf.square(modeloutput),1)
        
        return tfp.stats.percentile(tf.sqrt(distance), (1 - self.nu)*100)
 
    def loss_function(self, output, radius, tau):

        
        slack = tf.reduce_sum(tf.square(output), 1) - tf.square(radius)
        objective = tf.square(radius) + self.C * (1/tau)* tf.reduce_sum(tf.math.softplus(tau * slack))

        return objective


    @tf.function
    def train_model(self, data, radius, tau):
        with tf.GradientTape() as tape:

            output = self.SVDD_model(data, training=True)
            loss_train = self.loss_function( output, radius, tau)

        grads = tape.gradient(loss_train, (self.SVDD_model.trainable_variables))
        self.optimizer.apply_gradients(zip(grads, (self.SVDD_model.trainable_variables)))

        return loss_train
    
    def train(self, train_dataset , warm_up_n_epochs = 100, max_tau = 15):
        objective_break = False
   
        objective_list = []
        validation_loss_list = []

        
        pre_objective_loss = np.inf
        objective_loss = 0

        epoch = 0
        tau = 1
        for epoch in range(500):
        #while abs(pre_objective_loss - objective_loss) > 1e-6:
            epoch_loss_avg = tf.metrics.Mean()
            if (epoch >= warm_up_n_epochs):
                pre_objective_loss = objective_loss
            for batch, (batch_x) in enumerate(train_dataset):
                loss = self.train_model(batch_x, self.radius, tau)
                epoch_loss_avg(loss)
                if (epoch >= warm_up_n_epochs):
                    self.radius = self.get_radius(batch_x)
                
            objective_loss = epoch_loss_avg.result()
            #objective_list.append(objective_loss)
            ##改變tau
            if (epoch >= warm_up_n_epochs) and (epoch + 1) % 5 == 0 and tau < max_tau :
                tau += 1
            #epoch += 1
        #plt.plot(objective_list)
        #plt.show()
        self.SVDD_model.save_weights('model')
        np.save('radius',  self.radius.numpy())      
        
    def distance(self, data, load_model = True):
        if load_model:
            self.SVDD_model.load_weights('model')
            radius_end = np.load('radius.npy')

        output = self.SVDD_model(tf.convert_to_tensor(data), training=False)
        dis_square = tf.reduce_sum(tf.square(output), 1)                 
        dis_square = dis_square.numpy()
        
        return dis_square, radius_end
    
    def result_plot (self, dists, x_train_mix, radius ):
        r = radius**2
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(x_train_mix[dists<=r][:,0], x_train_mix[dists<=r][:,1],x_train_mix[dists<=r][:,2] ,s=0.9, label='normal')
        plt.show()
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(x_train_mix[dists<=r][:,0], x_train_mix[dists<=r][:,1],x_train_mix[dists<=r][:,2] ,s=0.9, label='normal')
        ax.scatter(x_train_mix[dists>=r][:,0], x_train_mix[dists>=r][:,1],x_train_mix[dists>=r][:,2] ,s=0.9, label='anomaly')
        plt.legend()
        plt.show()
    
    def train_result(self, x_train_mix, show_img = False, load_model = True):

        if show_img:
            dis_square, radius_end = self.distance(x_train_mix, load_model = load_model)
            #self.result_plot (dis_square, x_train_mix, radius_end )

        return radius_end
    
    def result(self, dist, y_test, radius_end, show_img = False):
        
        y_true = y_test.astype(int)
        scores = dist - radius_end ** 2
        y_prob = 1/dist
        auc = roc_auc_score(y_true, y_prob)
        confusionmatrix = confusion_matrix(y_true, scores<0)
        if show_img:
            print('AUC:', auc)
            print('confusion_matrix:')
            print(confusionmatrix)

        return auc, confusionmatrix 
    
    def test_result(self, x_test_mix, y_test_mix, show_img = False, load_model = True):
        
        dis_square, radius_end = self.distance(x_test_mix, load_model = load_model)
        dis_square += 1e-16
        test_auc, confusionmatrix = self.result(dis_square, y_test_mix, radius_end, show_img = show_img)
        if show_img :
            self.result_plot (dis_square, x_test_mix, radius_end )

        return test_auc, confusionmatrix

In [67]:
nu = 0.1
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [68]:
result = np.zeros((25, 3))

In [69]:
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    #ae_model.fit(x_train_mix, x_train_mix, epochs = 10, verbose = 0)
    ## 訓練 SVDD
    SVDD_model = make_model(ae_model)
    SVDD = SA_DeepSVDD_Trainer(nu, SVDD_model, n_samples)
    SVDD.train(train_dataset)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9187451739618406
epoch =  1 auc =  0.9496218855218854
epoch =  2 auc =  0.915223007856341
epoch =  3 auc =  0.9545938832772165
epoch =  4 auc =  0.8935084175084176
epoch =  5 auc =  0.8925050505050505
epoch =  6 auc =  0.923784736251403
epoch =  7 auc =  0.9464941638608306
epoch =  8 auc =  0.9315653198653198
epoch =  9 auc =  0.9535980920314253
epoch =  10 auc =  0.853364534231201
epoch =  11 auc =  0.9057710437710438
epoch =  12 auc =  0.9241085297418631
epoch =  13 auc =  0.9249643097643098
epoch =  14 auc =  0.9386181818181818
epoch =  15 auc =  0.9341545454545455
epoch =  16 auc =  0.9211580808080808
epoch =  17 auc =  0.8959332210998879
epoch =  18 auc =  0.9036285072951739
epoch =  19 auc =  0.9629611672278339
epoch =  20 auc =  0.8753247474747474
epoch =  21 auc =  0.9511136924803592
epoch =  22 auc =  0.9284701459034792
epoch =  23 auc =  0.9585739057239057
epoch =  24 auc =  0.9112030303030303


In [70]:
np.mean(result, 0)

array([12.        ,  0.92275949, 15.16902176])

In [71]:
np.std(result, 0)

array([7.21110255, 0.02673676, 0.25661894])

In [72]:
nu = 0.2
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [73]:
result = np.zeros((25, 3))
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    #ae_model.fit(x_train_mix, x_train_mix, epochs = 10, verbose = 0)
    ## 訓練 SVDD
    SVDD_model = make_model(ae_model)
    SVDD = SA_DeepSVDD_Trainer(nu, SVDD_model, n_samples)
    SVDD.train(train_dataset)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9149031954887219
epoch =  1 auc =  0.87521171679198
epoch =  2 auc =  0.9264736215538847
epoch =  3 auc =  0.9417192042606518
epoch =  4 auc =  0.9376811403508772
epoch =  5 auc =  0.9279263157894737
epoch =  6 auc =  0.8705616541353381
epoch =  7 auc =  0.9364637844611529
epoch =  8 auc =  0.9218034461152883
epoch =  9 auc =  0.9425971177944862
epoch =  10 auc =  0.91895
epoch =  11 auc =  0.8916946741854637
epoch =  12 auc =  0.9124177318295741
epoch =  13 auc =  0.9162941729323308
epoch =  14 auc =  0.9185791979949874
epoch =  15 auc =  0.8947364035087719
epoch =  16 auc =  0.8828087719298245
epoch =  17 auc =  0.8995000626566416
epoch =  18 auc =  0.946208395989975
epoch =  19 auc =  0.9272713659147869
epoch =  20 auc =  0.9276800751879699
epoch =  21 auc =  0.8882159774436091
epoch =  22 auc =  0.9256857769423561
epoch =  23 auc =  0.9324297619047619
epoch =  24 auc =  0.913442418546366


In [74]:
np.mean(result, 0)

array([12.        ,  0.91565024, 15.20458014])

In [75]:
np.std(result, 0)

array([7.21110255, 0.02100775, 0.24701374])

In [76]:
nu = 0.3
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [77]:
result = np.zeros((25, 3))
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    #ae_model.fit(x_train_mix, x_train_mix, epochs = 10, verbose = 0)
    ## 訓練 SVDD
    SVDD_model = make_model(ae_model)
    SVDD = SA_DeepSVDD_Trainer(nu, SVDD_model, n_samples)
    SVDD.train(train_dataset)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9415479761904761
epoch =  1 auc =  0.9027825952380951
epoch =  2 auc =  0.9169476428571429
epoch =  3 auc =  0.9208438571428571
epoch =  4 auc =  0.8915999523809525
epoch =  5 auc =  0.9069999523809525
epoch =  6 auc =  0.9450937619047619
epoch =  7 auc =  0.9482450952380952
epoch =  8 auc =  0.922454857142857
epoch =  9 auc =  0.8857462857142857
epoch =  10 auc =  0.9155732857142856
epoch =  11 auc =  0.8792046666666666
epoch =  12 auc =  0.8156025238095238
epoch =  13 auc =  0.9218631190476191
epoch =  14 auc =  0.941291476190476
epoch =  15 auc =  0.8913238095238096
epoch =  16 auc =  0.8721340000000001
epoch =  17 auc =  0.9063330476190476
epoch =  18 auc =  0.9306551666666668
epoch =  19 auc =  0.9196180714285713
epoch =  20 auc =  0.9495069523809523
epoch =  21 auc =  0.9103024285714286
epoch =  22 auc =  0.869942380952381
epoch =  23 auc =  0.8879666666666667
epoch =  24 auc =  0.9419558571428571


In [78]:
np.mean(result, 0)

array([12.        ,  0.90942142, 15.30332472])

In [79]:
np.std(result, 0)

array([7.21110255, 0.03032333, 0.27229235])